# Methods for achieving expert agreement of clinical images: the Medical Annotation and Review of Superpixels (MARk-SUP)
## Web-based expert annotation platform piloted for expert annotation and quantitative analysis of dermoscopic features

### *Online supplementary Jupyter notebook with analyses*

The cells in this notebook can be run in succession, or, as
desired, cell-by-cell.

**Please note:** the first five cells **MUST** be run in order
to establish the necessary variables in the workspace of the
python kernel!

In [ ]:
# imports
import os

import imageio
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import pyplot as plt
import numpy
import pandas as pd

%matplotlib inline

from isicarchive import font, func, imfunc
from isicarchive.api import IsicApi

# function for mean and sample STD
def mean_std(a:list, is_sample:bool=True):
    ddof = 1 if is_sample else 0
    return (numpy.mean(a), numpy.std(a, ddof=ddof))

In [ ]:
# settings
print_details = True # set to True for details in the print-out
print_fine_details = False # for even more details

heatmap_mix_colors = False # set to True for mixed rather than striped patches
heatmap_underlay_gray = 0.8 # remove this much color from images for heatmaps
heatmap_resize_output = 4096 # set heatmap output size (prior to montage)
heatmap_legend_font_size = 144.0 # legend font-size (default, prior to fitting)

overlap_compute_smcc = False # also compute smoothed-cross-correlation (masks)
overlap_colormap = 'Greys' # colormap for overlap confusion matrices
overlap_blow_up = 24 # blow-up factor (for each cell)
overlap_remove_cats = ['Nail lesions'] # remove these categories from detail

sp_minpairs = 3
sp_diag_minpairs = 2
sp_thresh_min = 2.0 / 3.0
sp_thresh_max = 3.0 / 2.0
sp_list_thresh = 0.5

calibri = font.Font('calibri') # font to use (currently only one available!)

# please change the username if you wish to re-run the notebook!
username = 'weberj3@mskcc.org'

# root folder for all ISIC related data
doc_folder = 'Z:\\10.Imaging Informatics\\'

# cache folder
cache_folder = doc_folder + 'ISIC' + os.sep + 'cache'

# show URL requests (for debugging purposes)
debug = False

In [ ]:
# instantiate API object
api = IsicApi(username, cache_folder=cache_folder, debug=debug)

In [ ]:
# study folder
study_folder = doc_folder + 'EASY' + os.sep + 'PILOT' + os.sep

# load study and data
study = api.study('ISIC Annotation Study - All Features')
study.cache_image_data()
study.load_annotations()

# load meta data
meta_data_url = ('https://raw.githubusercontent.com/neuroelf/' +
    'isicarchive/master/data/EASY_pilot_diagnoses.csv')
study.load_meta_data(meta_data_url, list_to_dict=True,
    dict_key='name', extract_key=['diagnosis', 'exemplar'])

# and create a dictionary mapping diagnosis to a list of images
diag_images = dict()
for (name, diag) in study.meta_data['diagnosis'].items():
    if not diag in diag_images:
        diag_images[diag] = []
    diag_images[diag].append(name)

# same for exemplar features
exem_images = dict()
for (name, exemplar) in study.meta_data['exemplar'].items():
    if not exemplar:
        continue
    if not exemplar in exem_images:
        exem_images[exemplar] = []
    exem_images[exemplar].append(name)

In [ ]:
# select only from users that completed the study
image_names = [img['name'] for img in study.images]
num_images = len(study.images)
users = [u for (u,c) in study.user_completion.items() if c==num_images]
study.select_annotations(users=users)
user_names = {u['_id']: u['name'].replace('User ', '') for u in study.users}
user_idx = dict()
for (idx, u) in enumerate(users):
    user_idx[u] = idx

# number of all annotations (across users, images, features)
total_features_annotations = sum(
    [len(a.features) for a in study.annotation_selection.values()])

# determine which features were used
selected_features = dict()
for annotation in study.annotation_selection.values():
    for feature in annotation.features:
        selected_features[feature] = True

# and create necessary lists
full_features_list = sorted(list([f['id'] for f in study.features]))
features_list = sorted(selected_features.keys())
num_features = len(features_list)
features_idx = dict()
for (feat_idx, feat_name) in enumerate(features_list):
    features_idx[feat_name] = feat_idx
category_list = sorted(list(set([v.split(' : ')[0] for v in features_list])))
num_categories = len(category_list)
category_idx = dict()
for (cat_idx, cat_name) in enumerate(category_list):
    category_idx[cat_name] = cat_idx

This is an optional cell (for heatmaps and overlap statistics; the
results of which are needed for some cells below, but the computation
takes a lot of time, so it's split out into a separate cell.

In [ ]:
# create heatmaps with default settings (if not yet done, about 30 minutes)
study_stats_file = study_folder + 'heatmap_stats.json.gz'
if not os.path.exists(study_stats_file):
    study_stats = study.image_heatmaps(study_folder, users=users,
        mix_colors=heatmap_mix_colors, underlay_gray=heatmap_underlay_gray, 
        resize_output=heatmap_resize_output, font_size=heatmap_legend_font_size)
else:
    study_stats = func.gzip_load_var(study_stats_file)

# compute feature overlap stats
overlap_stats_file = study_folder + 'overlap_features.npz'
if not os.path.exists(overlap_stats_file):
    overlap_stats = study.overlap_stats(users=users, 
        compute_smcc=overlap_compute_smcc)
    overlap_features_dice = overlap_stats[4]
    overlap_feat_cat_dice = overlap_stats[6]
    overlap_category_dice = overlap_stats[8]
    if overlap_compute_smcc:
        overlap_features_smcc = overlap_stats[10]
        overlap_feat_cat_smcc = overlap_stats[12]
        overlap_category_smcc = overlap_stats[14]
        numpy.savez(overlap_stats_file,
            overlap_features_dice=overlap_features_dice,
            overlap_feat_cat_dice=overlap_feat_cat_dice,
            overlap_category_dice=overlap_category_dice,
            overlap_features_smcc=overlap_features_smcc,
            overlap_feat_cat_smcc=overlap_feat_cat_smcc,
            overlap_category_smcc=overlap_category_smcc)
    else:
        numpy.savez(overlap_stats_file,
            overlap_features_dice=overlap_features_dice,
            overlap_feat_cat_dice=overlap_feat_cat_dice,
            overlap_category_dice=overlap_category_dice)
overlap_stats = numpy.load(overlap_stats_file)
overlap_features_dice = overlap_stats.get('overlap_features_dice')
overlap_feat_cat_dice = overlap_stats.get('overlap_feat_cat_dice')
overlap_category_dice = overlap_stats.get('overlap_category_dice')
if 'overlap_features_smcc' in overlap_stats.keys():
    overlap_compute_smcc = True
if overlap_compute_smcc:
    try:
        overlap_features_smcc = overlap_stats.get('overlap_features_smcc')
        overlap_feat_cat_smcc = overlap_stats.get('overlap_feat_cat_smcc')
        overlap_category_smcc = overlap_stats.get('overlap_category_smcc')
    except:
        overlap_compute_smcc = False

# select those annotations, and gather basic statistics
selected_annotations = study.select_annotations(users=users)

This gives some basic statistics of the study:

In [ ]:
print('{1:d} readers annotated the {2:d} dermoscopic images.'.format(
    study.name, len(users), len(study.images)))
if print_details:
    print(' ... per diagnosis:')
    for (diag, image_list) in diag_images.items():
        print(' - {0:d} in {1:s}'.format(len(image_list), diag))

In [ ]:
print('Out of {0:d} offered features, {1:d} were selected at least once.'.format(
    len(study.features), len(selected_features)))

In [ ]:
print('Each reader annotated an average of {0:.1f} features per lesion.'.format(
    float(total_features_annotations) / len(study.annotation_selection)))
print('In total, {0:d} feature annotations (markups) were made.'.format(
    total_features_annotations))

Some additional computations for complex statistics:

In [ ]:
# compute minimum and maximum number of features (per image/diagnosis)
(fmin, fmin_std, fmin_diag) = (85.0, 0.0, None)
(fmax, fmax_std, fmax_diag) = (0.0, 0.0, None)
if print_details:
    print('On average, images with diagnosis ...')
for diagnosis in sorted(diag_images.keys()):
    study.select_annotations(images=diag_images[diagnosis], users=users)
    ao = [a for a in study.annotation_selection.values()]
    fn = [None] * len(ao)
    for (idx,a) in enumerate(ao):
        fn[idx] = len(a.features)
    (m,s) = mean_std(fn)
    if m < fmin:
        (fmin, fmin_std, fmin_diag) = (m, s, diagnosis)
    if m > fmax:
        (fmax, fmax_std, fmax_diag) = (m, s, diagnosis)
    if print_details:
        print(' - "{0:s}" have {1:.2f} ± {2:.2f} annotations.'.format(
            diagnosis, m, s))
print('Based on the diagnosis, the number of features per image varied from')
print('{0:.2f} (±{1:.2f}) for {2:s} to {3:.2f} (±{4:.2f}) for {5:s}.'.format(
    fmin, fmin_std, fmin_diag, fmax, fmax_std, fmax_diag))

In [ ]:
# for each image, test whether all five raters agreed on one feature
feature_annotations = dict()
feature_annotation_stats = dict()
user_feature_stats = [[[] for n in range(num_images)]
    for u in range(len(user_idx))]
for feature in full_features_list:
    feature_annotations[feature] = dict()
    feature_annotation_stats[feature] = None
image_agreed = [False] * len(study.images)
image_agreed_features = [[] for l in range(len(study.images))]
image_orphan_features = []
for (idx, image) in enumerate(study.images):
    image_id = image['_id']
    image_name = image['name']
    study.select_annotations(images=[image_id], users=users)
    ao = [a for a in study.annotation_selection.values()]
    for feature in ao[0].features.keys():
        agreed = [False] * len(ao)
        feature_syns = api.feature_synonyms(feature)
        for (aidx, a) in enumerate(ao):
            for f in feature_syns:
                if f in a.features:
                    agreed[aidx] = True
                    break
        if all(agreed):
            image_agreed[idx] = True
            image_agreed_features[idx].append(feature)
    for a in ao:
        uidx = user_idx[a.user_id]
        for (feature,fc) in a.features.items():
            user_feature_stats[uidx][idx].append(
                '{0:d} ({1:d})'.format(features_idx[feature], len(fc['idx'])))
            if not feature in feature_annotations:
                feature_annotations[feature] = dict()
            if not image_name in feature_annotations[feature]:
                feature_annotations[feature][image_name] = 0
            feature_annotations[feature][image_name] += 1
for feature in full_features_list:
    features_annotated = [1 if v >= 3 else 0
        for v in feature_annotations[feature].values()]
    if not features_annotated:
        continue
    feature_agreed = sum(features_annotated)
    feature_annotation_stats[feature] = feature_agreed / len(
        feature_annotations[feature])
feature_annotation_levels = numpy.asarray(
    [v for v in feature_annotation_stats.values() if not v is None])
if print_details:
    print('Feature-in-image agreements:')
total_agreements = 0
for (image, a, af) in zip(study.images, image_agreed, image_agreed_features):
    if a:
        total_agreements += len(af)
        image_name = image['name']
        image_diag = study.meta_data['diagnosis'][image_name]
        if print_details:
            print(' - {0:s} ({1:s}): {2:s}'.format(image_name,
                image_diag, ', '.join(af)))
print('There were a total of {0:d} feature-in-image agreements.'.format(
    total_agreements))
print(('These were reached in a total of {0:d} images.').format(
    numpy.sum(image_agreed)))
gold_standard_50 = [feature for (feature, a_level) in feature_annotation_stats.items()
    if a_level and a_level >= 0.5]
print('The gold standard (of 60% agreement) was reached for ' +
    '{0:d}'.format(len(gold_standard_50)) + ' features:')
for feature in gold_standard_50:
    print(' - {0:.1f}% cases for feature "{1:s}"'.format(
        100.0 * feature_annotation_stats[feature], feature))

In [ ]:
# count features, including orphans
feature_markups = dict()
feature_in_images = dict()
agreed_2 = dict()
agreed_3 = dict()
agreed_4 = dict()
agreed_5 = dict()
for feat_name in features_list:
    feature_markups[feat_name] = 0
    feature_in_images[feat_name] = set()
    agreed_2[feat_name] = []
    agreed_3[feat_name] = []
    agreed_4[feat_name] = []
    agreed_5[feat_name] = []
orphans = 0
orphan_image_features = []
orphan_features = dict()
benign_diag_images = []
benign_diag_features = dict()
malignant_diag_images = []
malignant_diag_features = dict()
unclear_diag_images = []
unclear_diag_features = dict()
user_diagnosis = {u: [] for u in users}
for (idx, image) in enumerate(study.images):
    image_id = image['_id']
    image_name = image['name']
    study.select_annotations(images=[image_id], users=users)
    ao = [a for a in study.annotation_selection.values()]
    diag_benign = False
    diag_malignant = False
    diag_unclear = False
    diag_unknown = False
    user_diag = {u: False for u in users}
    for a in ao:
        user_diag[a.user_id] = True
        try:
            image_diag = a.responses['Benign or Malignant']
            user_diagnosis[a.user_id].append(image_diag)
            if image_diag[0] == 'B':
                diag_benign = True
            elif image_diag[0] == 'M':
                diag_malignant = True
        except:
            user_diagnosis[a.user_id].append('Unsure')
            diag_unknown = True
            pass
    for (u, u_d) in user_diag.items():
        if not u_d:
            user_diagnosis[u].append(None)
    if diag_benign and diag_malignant:
        diag_unclear = True
        unclear_diag_images.append(image_id)
    elif diag_benign and not diag_unknown:
        benign_diag_images.append(image_id)
    elif diag_malignant and not diag_unknown:
        malignant_diag_images.append(image_id)
    for feature in features_list:
        fcount = 0
        for a in ao:
            if feature in a.features:
                fcount += 1
                if diag_unclear:
                    if not feature in unclear_diag_features:
                        unclear_diag_features[feature] = []
                    unclear_diag_features[feature].append(image_id)
                elif diag_benign and not diag_unknown:
                    if not feature in benign_diag_features:
                        benign_diag_features[feature] = []
                    benign_diag_features[feature].append(image_id)
                elif diag_malignant and not diag_unknown:
                    if not feature in malignant_diag_features:
                        malignant_diag_features[feature] = []
                    malignant_diag_features[feature].append(image_id)
        if fcount > 1:
            agreed_2[feature].append(image_name)
        if fcount > 2:
            agreed_3[feature].append(image_name)
        if fcount > 3:
            agreed_4[feature].append(image_name)
        if fcount > 4:
            agreed_5[feature].append(image_name)
    for (aidx, a) in enumerate(ao):
        for feature in a.features:
            feature_markups[feature] += 1
            feature_in_images[feature].add(image_name)
            is_orphan = True
            feature_syns = api.feature_synonyms(feature)
            for (aidx2, a2) in enumerate(ao):
                if aidx == aidx2:
                    continue
                for feature2 in a2.features:
                    if feature2 in feature_syns:
                        is_orphan = False
                        break
                if not is_orphan:
                    break
            if is_orphan:
                orphans += 1
                orphan_image_features.append(feature + ' in ' + 
                    image_name + ' by ' + a.user['name'])
                if not feature_syns[0] in orphan_features:
                    orphan_features[feature_syns[0]] = 0
                orphan_features[feature_syns[0]] += 1
print('An orphan observation of a feature occurred {0:d} times.'.format(
    orphans))
if print_details:
    print('{0:d} orphan features were selected:'.format(orphans))
    for orphaned in orphan_image_features:
        print(' - ' + orphaned)
    print('Presenting by list of features:')
    for feature in sorted(orphan_features.keys()):
        print(' - {0:-2d} times "{1:s}"'.format(
            orphan_features[feature], feature))
    print('')
print('Out of the {0:d} images, {1:d} were given mixed diagnoses.'.format(
    num_images, len(unclear_diag_images)))

In [ ]:
# how many exemplar feature annotations for each image with an exemplar?
gold_standard_examplars = 0
total_exemplar_images = 0
total_found_direct = 0
total_found_list = []
total_found_category = 0
total_found_specific = 0
for exemplar in sorted(exem_images.keys()):
    images = exem_images[exemplar]
    if print_details:
        print('Exemplar "{0:s}" with {1:d} images:'.format(
            exemplar, len(images)))
    for image in images:
        total_exemplar_images += 1
        imag_diag = study.meta_data['diagnosis'][image]
        study.select_annotations(images=[image], users=users)
        ao = [a for a in study.annotation_selection.values()]
        found_direct = 0
        found_category = 0
        found_specific = 0
        for (idx,a) in enumerate(ao):
            if exemplar in a.features:
                found_direct += 1
                found_category += 1
                found_specific +=1
                continue
            found_at_all = False
            for feature in a.features:
                feature = feature.split(' : ')
                if exemplar[0:len(feature[0])] == feature[0]:
                    found_category += 1
                    found_at_all = True
                if feature[-1] in exemplar:
                    found_specific += 1
                    found_at_all = True
                if found_at_all:
                    break
        if found_direct >= 3:
            gold_standard_examplars += 1
        if found_direct == len(ao):
            total_found_direct += 1
            total_found_list.append(image)
        if found_specific == len(ao):
            total_found_specific += 1
        if found_category == len(ao):
            total_found_category += 1
        if print_details:
            print((' - {0:s} ({1:s}) has {2:d} annotations; ' +
                   '{3:d}, {4:d}, and {5:d} with the full, category, ' +
                   'and specific exemplar').format(image, imag_diag,
                    len(ao), found_direct, found_category, found_specific))
print(('Out of {0:d} images with an exemplar, the gold standard for the\n' +
    'exemplar image (60% agreement) was reached {1:d} times (~{2:d}%).').format(
    total_exemplar_images, gold_standard_examplars,
    int(100*gold_standard_examplars/total_exemplar_images)))
print('And out of these same images, all readers identified this')
print('feature {0:d} times, with {1:d} category and {2:d} specific hits.'.format(
    total_found_direct, total_found_category, total_found_specific))
if print_details:
    print('Direct hits:')
    for image in total_found_list:
        print(' - {0:s} ({1:s}, {2:s})'.format(image,
            study.meta_data['diagnosis'][image],
            study.meta_data['exemplar'][image]))

In [ ]:
# compute the superpixel-wise agreement
# (i.e. where 2 or more readers all agreed or any reader
# disagreed with the feature in the same superpixel)
print_fine_details=False
if print_details:
    print('Per-image super-pixel agreement:')
image_sp_stats = dict()
gs_agreed = 0
total_agreed = 0
total_disagreed = 0
sp_agreed_features = dict()
for (image_name, image_stats) in study_stats.items():
    orphans = 0
    agreed = 0
    disagreed = 0
    for (spidx, sp_stats) in image_stats['sp'].items():
        sp_keys = list(sp_stats.keys())
        for sp_key in sp_keys:
            if not sp_key in sp_agreed_features:
                sp_agreed_features[sp_key] = {
                    'agreed': 0,
                    'disagreed': 0,
                    'gs_agreed': 0,
                    'overlap': dict(),
                }
        if len(sp_keys) == 1:
            if len(sp_stats[sp_keys[0]]) == 1:
                orphans += 1
            else:
                agreed += 1
                sp_agreed_features[sp_keys[0]]['agreed'] += 1
        else:
            disagreed += 1
            for sp_key in sp_keys:
                sp_agreed_features[sp_key]['disagreed'] += 1
                if len(sp_stats[sp_key]) >= 3:
                    sp_agreed_features[sp_key]['gs_agreed'] += 1
                for o_key in sp_keys:
                    if sp_key != o_key:
                        if not o_key in sp_agreed_features[sp_key]['overlap']:
                            sp_agreed_features[sp_key]['overlap'][o_key] = 0
                        sp_agreed_features[sp_key]['overlap'][o_key] += 1
    total_agreed += agreed
    total_disagreed += disagreed
    image_sp_stats[image_name] = {
        'orphans': orphans, 'agreed': agreed, 'disagreed': disagreed}
    if print_details:
        print(' - {0:s} {1:-4.1f}% agreed ({2:d} SPs, w/o orphans; {3:s})'.format(
            image_name, 100.0 * float(agreed) / float(agreed + disagreed),
            agreed + disagreed, study.meta_data['diagnosis'][image_name]))
print('Total agreement {0:-4.1f}% ({1:d} out of {2:d} total superpixels)'.format(
    100.0 * float(total_agreed) / float(total_agreed + total_disagreed),
    total_agreed, total_agreed + total_disagreed))
if print_details:
    print('Agreement by feature:')
    for sp_key in sorted(sp_agreed_features.keys()):
        sp_agreement = sp_agreed_features[sp_key]
        print(' - {0:-4.1f}% GS / {1:-4.1f}% full agreement for {2:s} ({3:d} SPs)'.format(
            100.0 * float(sp_agreement['gs_agreed']) /
            float(sp_agreement['agreed'] + sp_agreement['disagreed']),
            100.0 * float(sp_agreement['agreed']) /
            float(sp_agreement['agreed'] + sp_agreement['disagreed']),
            sp_key, sp_agreement['agreed'] + sp_agreement['disagreed']))
        if print_fine_details:
            for o_key in sorted(sp_agreement['overlap'].keys()):
                print('   - {0:-3d} overlapped with {1:s}'.format(
                    sp_agreement['overlap'][o_key], o_key))

In [ ]:
# compute feature-based super-pixel co-occurrences
benign_features = dict()
malignant_features = dict()
category_sp_hits = numpy.zeros((num_categories, num_categories,))
category_sp_miss = numpy.zeros((num_categories, num_categories,))
category_sp_basis = numpy.zeros((num_categories, num_categories,))
feature_sp_hits = numpy.zeros((num_features, num_features,))
feature_sp_miss = numpy.zeros((num_features, num_features,))
feature_sp_basis = numpy.zeros((num_features, num_features,))
feat_cat_sp_hits = numpy.zeros((num_features, num_categories,))
feat_cat_sp_miss = numpy.zeros((num_features, num_categories,))
feat_cat_sp_basis = numpy.zeros((num_features, num_categories,))
sp_pairs_overlap = dict()
sp_pairs_overlap_diag = dict()
sp_thresh_min = 2.0 / 3.0
sp_thresh_max = 3.0 / 2.0
for (idx, image) in enumerate(study.images):
    image_id = image['_id']
    study.select_annotations(images=[image_id], users=users)
    ao = [a for a in study.annotation_selection.values()]
    for (aidx, a) in enumerate(ao):
        for (feature, fcont) in a.features.items():
            feature_cat = feature.split(' : ')[0]
            try:
                feature_diag = (a.responses['Benign or Malignant']
                    + '>' + feature)
                if feature_diag[0] == 'B':
                    if not feature in benign_features:
                        benign_features[feature] = []
                    benign_features[feature].append(image_id)
                elif feature_diag[0] == 'M':
                    if not feature in malignant_features:
                        malignant_features[feature] = []
                    malignant_features[feature].append(image_id)
            except:
                feature_diag = 'Unsure>' + feature
            fnum = features_idx[feature]
            cnum = category_idx[feature_cat]
            fidx = set(fcont['idx'])
            didx = float(len(fidx))
            for (aidx2, a2) in enumerate(ao):
                if aidx == aidx2:
                    continue
                for (feature2, fcont2) in a2.features.items():
                    feature2_cat = feature2.split(' : ')[0]
                    try:
                        feature2_diag = (a2.responses['Benign or Malignant']
                            + '>' + feature2)
                    except:
                        feature2_diag = 'Unsure>' + feature2
                    fnum2 = features_idx[feature2]
                    cnum2 = category_idx[feature2_cat]
                    fidx2 = set(fcont2['idx'])
                    didx2 = float(len(fidx2))
                    finter = fidx.intersection(fidx2)
                    finterlen = float(len(finter))
                    category_sp_hits[cnum,cnum2] += finterlen
                    category_sp_hits[cnum2,cnum] += finterlen
                    category_sp_miss[cnum,cnum2] += didx - finterlen
                    category_sp_miss[cnum2,cnum] += didx2 - finterlen
                    category_sp_basis[cnum,cnum2] += didx
                    category_sp_basis[cnum2,cnum] += didx2
                    feature_sp_hits[fnum,fnum2] += finterlen
                    feature_sp_hits[fnum2,fnum] += finterlen
                    feature_sp_miss[fnum,fnum2] += didx - finterlen
                    feature_sp_miss[fnum2,fnum] += didx2 - finterlen
                    feature_sp_basis[fnum,fnum2] += didx
                    feature_sp_basis[fnum2,fnum] += didx2
                    feat_cat_sp_hits[fnum,cnum2] += finterlen
                    feat_cat_sp_hits[fnum2,cnum] += finterlen
                    feat_cat_sp_miss[fnum,cnum2] += didx - finterlen
                    feat_cat_sp_miss[fnum2,cnum] += didx2 - finterlen
                    feat_cat_sp_basis[fnum,cnum2] += didx
                    feat_cat_sp_basis[fnum2,cnum] += didx2
                    if aidx >= aidx2 or feature == feature2:
                        continue
                    if didx < 3 or didx2 < 3:
                        continue
                    dfac = didx / didx2
                    if dfac < sp_thresh_min or dfac > sp_thresh_max:
                        continue
                    if feature < feature2:
                        fp = feature + ' *and* ' + feature2
                    else:
                        fp = feature2 + ' *and* ' + feature
                    fp_dice = 2 * finterlen / (didx + didx2)
                    if not fp in sp_pairs_overlap:
                        sp_pairs_overlap[fp] = []
                    sp_pairs_overlap[fp].append(fp_dice)
                    if feature_diag < feature2_diag:
                        fp = feature_diag + ' *and* ' + feature2_diag
                    else:
                        fp = feature2_diag + ' *and* ' + feature_diag
                    if not fp in sp_pairs_overlap_diag:
                        sp_pairs_overlap_diag[fp] = []
                    sp_pairs_overlap_diag[fp].append(fp_dice)
category_sp_basis[category_sp_basis == 0.0] = 1.0
category_sp_hits = category_sp_hits / category_sp_basis
category_sp_miss = category_sp_miss / category_sp_basis
feature_sp_basis[feature_sp_basis == 0.0] = 1.0
feature_sp_hits = feature_sp_hits / feature_sp_basis
feature_sp_miss = feature_sp_miss / feature_sp_basis
feat_cat_sp_basis[feat_cat_sp_basis == 0.0] = 1.0
feat_cat_sp_hits = feat_cat_sp_hits / feat_cat_sp_basis
feat_cat_sp_miss = feat_cat_sp_miss / feat_cat_sp_basis
sp_pairs_overlap_diagsame = {
    fp: fp_list for (fp, fp_list) in sp_pairs_overlap_diag.items()}
for fp in list(sp_pairs_overlap.keys()):
    if len(sp_pairs_overlap[fp]) < sp_minpairs:
        sp_pairs_overlap.pop(fp)
for fp in list(sp_pairs_overlap_diag.keys()):
    if (len(sp_pairs_overlap_diag[fp]) < sp_diag_minpairs or
        not 'Benign' in fp or not 'Malignant' in fp):
        sp_pairs_overlap_diag.pop(fp)
for fp in list(sp_pairs_overlap_diagsame.keys()):
    if (len(sp_pairs_overlap_diagsame[fp]) < sp_diag_minpairs or
        'Unsure' in fp or
        ('Benign' in fp and 'Malignant' in fp)):
        sp_pairs_overlap_diagsame.pop(fp)
sp_pairs_overlap = list(reversed(sorted(sp_pairs_overlap.items(),
    key=lambda x: numpy.mean(x[1]))))
sp_pairs_overlap_diag = list(reversed(sorted(sp_pairs_overlap_diag.items(),
    key=lambda x: numpy.mean(x[1]))))
sp_pairs_overlap_diagsame = list(reversed(sorted(sp_pairs_overlap_diagsame.items(),
    key=lambda x: numpy.mean(x[1]))))
sp_pairs_highoverlap = sum(
    [1 if numpy.mean(sp[1]) >= sp_list_thresh else 0 for sp in sp_pairs_overlap])
sp_pairs_highoverlap_diag = sum(
    [1 if numpy.mean(sp[1]) >= sp_list_thresh else 0 for sp in sp_pairs_overlap_diag])
sp_pairs_highoverlap_diagsame = sum(
    [1 if numpy.mean(sp[1]) >= sp_list_thresh else 0
     for sp in sp_pairs_overlap_diagsame])
print(('There are {0:d} pairs of discordant features with ' +
    'high (> {1:.2f} DICE) overlap:').format(sp_pairs_highoverlap, sp_list_thresh))
if print_details:
    for sp in sp_pairs_overlap:
        sp_mean = numpy.mean(sp[1])
        if sp_mean < sp_list_thresh:
            break
        print(' - {0:.1f}% ({1:d} pairs) for features {2:s}'.format(
            100.0 * sp_mean, len(sp[1]), sp[0]))
    print('')
print(('For different diagnoses, there are {0:d} pairs of discordant features with ' +
    'high (> {1:.2f} DICE) overlap:').format(sp_pairs_highoverlap_diag, sp_list_thresh))
if print_details:
    for sp in sp_pairs_overlap_diag:
        sp_mean = numpy.mean(sp[1])
        if sp_mean < sp_list_thresh:
            break
        print(' - {0:.1f}% ({1:d} pairs) for features {2:s}'.format(
            100.0 * sp_mean, len(sp[1]), sp[0]))
    print('')
print(('For equal diagnoses, there are {0:d} pairs of discordant features with ' +
    'high (> {1:.2f} DICE) overlap:').format(sp_pairs_highoverlap_diagsame, sp_list_thresh))
if print_details:
    for sp in sp_pairs_overlap_diagsame:
        sp_mean = numpy.mean(sp[1])
        if sp_mean < sp_list_thresh:
            break
        print(' - {0:.1f}% ({1:d} pairs) for features {2:s}'.format(
            100.0 * sp_mean, len(sp[1]), sp[0]))
    print('')
disdiag_sp_overlap = []
for (feature, feature_images) in benign_features.items():
    if feature in malignant_features:
        continue
    for image_id in feature_images:
        for (feature2, feature2_images) in malignant_features.items():
            if not image_id in feature2_images:
                continue
            study.select_annotations(images=[image_id], users=users,
                features=[feature, feature2])
            ao = [a for a in study.annotation_selection.values()]
            for (aidx, a) in enumerate(ao):
                if not feature in a.features:
                    continue
                fcont = a.features[feature]
                fidx = set(fcont['idx'])
                didx = float(len(fidx))
                for (aidx2, a2) in enumerate(ao):
                    if aidx == aidx2:
                        continue
                    if not feature2 in a2.features:
                        continue
                    fcont2 = a2.features[feature2]
                    fidx2 = set(fcont2['idx'])
                    didx2 = float(len(fidx2))
                    finter = fidx.intersection(fidx2)
                    finterlen = float(len(finter))
                    fp_dice = 2 * finterlen / (didx + didx2)
                    if fp_dice < sp_list_thresh:
                        continue
                    disdiag_sp_overlap.append({
                        'Image': a.image['name'],
                        'Reader1': user_names[a.user_id],
                        'Reader2': user_names[a2.user_id],
                        'Benign': feature,
                        'Malignant': feature2,
                        'DICE': fp_dice,
                    })
for (feature, feature_images) in malignant_features.items():
    if feature in benign_features:
        continue
    for image_id in feature_images:
        for (feature2, feature2_images) in benign_features.items():
            if not image_id in feature2_images:
                continue
            study.select_annotations(images=[image_id], users=users,
                features=[feature, feature2])
            ao = [a for a in study.annotation_selection.values()]
            for (aidx, a) in enumerate(ao):
                if not feature in a.features:
                    continue
                fcont = a.features[feature]
                fidx = set(fcont['idx'])
                didx = float(len(fidx))
                for (aidx2, a2) in enumerate(ao):
                    if aidx == aidx2:
                        continue
                    if not feature2 in a2.features:
                        continue
                    fcont2 = a2.features[feature2]
                    fidx2 = set(fcont2['idx'])
                    didx2 = float(len(fidx2))
                    finter = fidx.intersection(fidx2)
                    finterlen = float(len(finter))
                    fp_dice = 2 * finterlen / (didx + didx2)
                    if fp_dice < sp_list_thresh:
                        continue
                    disdiag_sp_overlap.append({
                        'Image': a.image['name'],
                        'Reader1': user_names[a2.user_id],
                        'Reader2': user_names[a.user_id],
                        'Benign': feature2,
                        'Malignant': feature,
                        'DICE': fp_dice,
                    })

# Superpixel explanation (image)

Superpixels are an automatic segmentation performed by the ISIC Archive,
which then allows annotators (readers) to select specific parcels of an
image.

In [ ]:
# create a heatmap with default settings (User: Liopyris)
sample_image = api.image('ISIC_0016094')
sample_image.load_image_data()
sample_data = sample_image.data
sample_image.mark_superpixels()
sp_data = sample_image.data
sample_image.clear_data()
(heatmap, stats) = study.image_heatmap('ISIC_0016094',
    mix_colors=False,underlay_gray=0.8,users=['578e64b09fc3c10d6fd12e4f'])
sp_image = imfunc.image_mix(sp_data, heatmap)
sp_comparison = numpy.concatenate((sample_data, sp_image), axis=1)
api.write_image(sp_comparison, study_folder + 'ISIC_0016094+hm_w_sp.png')
api.show_image_in_notebook(sp_comparison, max_size=1024)

## Heatmaps

The images below show (1) a demonstration of people using
different terms for the same feature, (2) people agreeing
agreeing on one specific term (but not others), and (3)
everybody agreeing on the (singular) term in an image.

In [ ]:
# show three images (heatmaps) of the study with agreement examples
print('(1)')
(heatmap, heatmap_stats) = study.image_heatmap('ISIC_0015549',
    users=users, underlay_gray=heatmap_underlay_gray,
    mix_colors=heatmap_mix_colors, resize_output=4096)
api.show_image_in_notebook(heatmap, max_size=1024)
print('(2)')
(heatmap, heatmap_stats) = study.image_heatmap('ISIC_0016094',
    users=users, underlay_gray=heatmap_underlay_gray,
    mix_colors=heatmap_mix_colors, resize_output=4096)
api.show_image_in_notebook(heatmap, max_size=1024)
print('(3)')
(heatmap, heatmap_stats) = study.image_heatmap('ISIC_0016128',
    users=users, underlay_gray=heatmap_underlay_gray,
    mix_colors=heatmap_mix_colors, resize_output=4096)
api.show_image_in_notebook(heatmap, max_size=1024)

## Feature overlap/confusion figure

The image prepared with the code below shows the matrix of feature overlap,
and thus confusability; collapsed across images/diagnoses.

In [ ]:
# get the color lookup map
color_norm = mpl.colors.Normalize(vmin=0.0, vmax=1.0)
color_map = cm.ScalarMappable(norm=color_norm, cmap=overlap_colormap)

overlap_stats = numpy.load(overlap_stats_file)
overlap_features_dice = overlap_stats.get('overlap_features_dice')
overlap_feat_cat_dice = overlap_stats.get('overlap_feat_cat_dice')
overlap_category_dice = overlap_stats.get('overlap_category_dice')

# remove categories from display
remove_cat_lines = []
for c in overlap_remove_cats:
    if c in category_list:
        remove_cat_lines.append(category_list.index(c))
remove_cat_lines = reversed(sorted(remove_cat_lines))
for c in remove_cat_lines:
    category_sp_hits = numpy.delete(category_sp_hits, c, axis=0)
    category_sp_hits = numpy.delete(category_sp_hits, c, axis=1)
    feat_cat_sp_hits = numpy.delete(feat_cat_sp_hits, c, axis=1)
    overlap_category_dice = numpy.delete(overlap_category_dice, c, axis=0)
    overlap_category_dice = numpy.delete(overlap_category_dice, c, axis=1)
    overlap_feat_cat_dice = numpy.delete(overlap_feat_cat_dice, c, axis=1)
    category_list.pop(c)

# process data prior to creating the image
white_val = numpy.uint8(255)
feature_sp_rgb = numpy.trunc(255.0 *
    color_map.to_rgba(feature_sp_hits)[:,:,0:3]).astype(numpy.uint8)
feat_cat_sp_rgb = numpy.trunc(255.0 *
    color_map.to_rgba(feat_cat_sp_hits)[:,:,0:3]).astype(numpy.uint8)
category_sp_rgb = numpy.trunc(255.0 *
    color_map.to_rgba(category_sp_hits)[:,:,0:3]).astype(numpy.uint8)
overlap_features_null = numpy.isnan(overlap_features_dice)
overlap_feat_cat_null = numpy.isnan(overlap_feat_cat_dice)
overlap_category_null = numpy.isnan(overlap_category_dice)
overlap_features_dice[overlap_features_null] = 0
overlap_feat_cat_dice[overlap_feat_cat_null] = 0
overlap_category_dice[overlap_category_null] = 0
overlap_features_rgb = numpy.trunc(255.0 *
    color_map.to_rgba(overlap_features_dice[:,:,0])[:,:,0:3]).astype(numpy.uint8)
overlap_features_rgb[numpy.repeat(overlap_features_null[:,:,0:1], 3, axis=2)] = 255
overlap_feat_cat_rgb = numpy.trunc(255.0 *
    color_map.to_rgba(overlap_feat_cat_dice[:,:,0])[:,:,0:3]).astype(numpy.uint8)
overlap_feat_cat_rgb[numpy.repeat(overlap_feat_cat_null[:,:,0:1], 3, axis=2)] = 255
overlap_category_rgb = numpy.trunc(255.0 *
    color_map.to_rgba(overlap_category_dice[:,:,0])[:,:,0:3]).astype(numpy.uint8)
overlap_category_rgb[numpy.repeat(overlap_category_null[:,:,0:1], 3, axis=2)] = 255

# add spacers spacers
f1v = white_val * numpy.ones((overlap_category_dice.shape[0], 2, 3,),
    dtype=numpy.uint8)
f2v = white_val * numpy.ones((overlap_features_dice.shape[0], 2, 3,),
    dtype=numpy.uint8)
blank_space = white_val * numpy.ones(
    (overlap_category_dice.shape[0], overlap_features_dice.shape[0], 3,),
    dtype=numpy.uint8)
hits_top_image = numpy.concatenate(
    (category_sp_rgb, f1v, blank_space), axis=1)
hits_bottom_image = numpy.concatenate(
    (feat_cat_sp_rgb, f2v, feature_sp_rgb), axis=1)
dice_top_image = numpy.concatenate(
    (overlap_category_rgb, f1v, blank_space), axis=1)
dice_bottom_image = numpy.concatenate(
    (overlap_feat_cat_rgb, f2v, overlap_features_rgb), axis=1)
f1h = white_val * numpy.ones((2, dice_top_image.shape[1], 3,), dtype=numpy.uint8)
hits_full_image = numpy.concatenate((hits_top_image, f1h, hits_bottom_image), axis=0)
dice_full_image = numpy.concatenate((dice_top_image, f1h, dice_bottom_image), axis=0)

# blow up by factor 24 (for text attribution)
detail_blow_up = 5 * overlap_blow_up
hits_detail_image = numpy.repeat(numpy.repeat(
    hits_full_image[0:len(category_list), 0:len(category_list), :],
    detail_blow_up, axis=0), detail_blow_up, axis=1)
hits_full_image = numpy.repeat(numpy.repeat(hits_full_image, overlap_blow_up, axis=0),
    overlap_blow_up, axis=1)
dice_detail_image = numpy.repeat(numpy.repeat(
    dice_full_image[0:len(category_list), 0:len(category_list), :],
    detail_blow_up, axis=0), detail_blow_up, axis=1)
dice_full_image = numpy.repeat(numpy.repeat(dice_full_image, overlap_blow_up, axis=0),
    overlap_blow_up, axis=1)

# determine line positions and paint lines
line_pos = []
cat_name = category_list[0]
for (idx,feature) in enumerate(features_list):
    if not cat_name in feature:
        line_pos.append(idx)
        cat_name = feature.split(' : ')[0]
line_offset = overlap_blow_up * (len(category_list) + 2) - 1
for lp in line_pos:
    line_xy = line_offset + overlap_blow_up * lp
    hits_full_image[line_xy:line_xy+2,:,:] = 0
    hits_full_image[line_offset:, line_xy:line_xy+2,:] = 0
    dice_full_image[line_xy:line_xy+2,:,:] = 0
    dice_full_image[line_offset:, line_xy:line_xy+2,:] = 0

# create text images
category_text = calibri.set_line(category_list, fsize=overlap_blow_up-1)
category_text_length = [v for v in map(lambda x: x.shape[1], category_text)]
detail_text = calibri.set_line(category_list, fsize=(3*overlap_blow_up))
detail_text_length = [v for v in map(lambda x: x.shape[1], detail_text)]
detail_max_length = max(detail_text_length)
features_text = calibri.set_line(features_list, fsize=overlap_blow_up-1)
features_text_length = [v for v in map(lambda x: x.shape[1], features_text)]
features_max_length = max(features_text_length)
detail_timage_x = detail_max_length + 4 * overlap_blow_up
detail_timage = white_val * numpy.ones(
    (dice_detail_image.shape[0], detail_timage_x, 3, ), dtype=numpy.uint8)
text_image_y = overlap_blow_up * (2 + len(category_text) + len(features_text))
text_image_x = features_max_length + 2 * overlap_blow_up
text_image = white_val * numpy.ones(
    (text_image_y, text_image_x, 3,), dtype=numpy.uint8)
for (idx, cat_name) in enumerate(category_list):
    line_w = category_text_length[idx]
    line_text = white_val - numpy.repeat(
        category_text[idx].reshape((overlap_blow_up, line_w, 1,)), 3, axis=2)
    line_xy = idx * overlap_blow_up
    line_x = overlap_blow_up + (features_max_length - line_w)
    text_image[line_xy:line_xy+overlap_blow_up,line_x:line_x+line_w,:] = line_text
    line_w = detail_text_length[idx]
    line_text = white_val - numpy.repeat(detail_text[idx].reshape(
        (detail_text[idx].shape[0], line_w, 1,)), 3, axis=2)
    line_xy = (idx * 5 + 1) * overlap_blow_up
    line_x = 2 * overlap_blow_up + (detail_max_length - line_w)
    detail_timage[line_xy:line_xy+line_text.shape[0],line_x:line_x+line_w,:] = line_text
for (idx, feat_name) in enumerate(features_list):
    line_w = features_text_length[idx]
    line_text = white_val - numpy.repeat(
        features_text[idx].reshape((overlap_blow_up, line_w, 1,)), 3, axis=2)
    line_xy = (len(category_list) + 2 + idx) * overlap_blow_up
    line_x = overlap_blow_up + (features_max_length - line_w)
    text_image[line_xy:line_xy+overlap_blow_up, line_x:line_x+line_w,:] = line_text

# put everything together
hits_detail_image = numpy.concatenate((detail_timage, hits_detail_image), axis=1)
dice_detail_image = numpy.concatenate((detail_timage, dice_detail_image), axis=1)
hits_full_image = numpy.concatenate(
    (hits_full_image, imfunc.image_rotate(text_image, 'left')), axis=0)
dice_full_image = numpy.concatenate(
    (dice_full_image, imfunc.image_rotate(text_image, 'left')), axis=0)
text_filler = white_val * numpy.ones((text_image_x, text_image_x, 3,),
    dtype=numpy.uint8)
text_image = numpy.concatenate((text_image, text_filler), axis=0)
hits_full_image = numpy.concatenate((text_image, hits_full_image), axis=1)
dice_full_image = numpy.concatenate((text_image, dice_full_image), axis=1)

# write and show image
api.write_image(hits_detail_image, study_folder + 'EASY_PILOT_sp-hits_confusion_markup_cat.png')
api.write_image(dice_detail_image, study_folder + 'EASY_PILOT_DICE_confusion_markup_cat.png')
api.write_image(hits_full_image, study_folder + 'EASY_PILOT_sp-hits_confusion_markup_full.png')
api.write_image(dice_full_image, study_folder + 'EASY_PILOT_DICE_confusion_markup_full.png')
detail_combined = numpy.concatenate((hits_detail_image, dice_detail_image), axis=1)
api.show_image_in_notebook(detail_combined, max_size=1024)
api.show_image_in_notebook(hits_full_image, max_size=1024)
api.show_image_in_notebook(dice_full_image, max_size=1024)

# Supplementary data

## Table of ISIC images

In [ ]:
S1_dict = {
    'ISIC name': image_names,
    'ISIC imageId': [img['_id'] for img in study.images],
    'diagnosis': [study.meta_data['diagnosis'][img['name']] for img in study.images],
    'exemplar feature': [study.meta_data['exemplar'][img['name']] for img in study.images],
}
for u in users:
    S1_dict['diag_' + user_names[u]] = user_diagnosis[u]
S1_study_images = pd.DataFrame.from_dict(S1_dict)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(S1_study_images)

## Tables of dermoscopic features

In [ ]:
S2_dermoscopic_features = pd.DataFrame.from_dict({
    'Dermoscopic Feature': full_features_list,
    'Total observations': [feature_markups[f] if f in feature_markups else 0
        for f in full_features_list],
    'In total images': [len(feature_in_images[f]) if f in feature_in_images else 0
        for f in full_features_list],
    'Orphans': [orphan_features[f] if f in orphan_features else 0
        for f in full_features_list],
    '2-RA': [len(agreed_2[f]) if f in agreed_2 else 0
        for f in full_features_list],
    'GS (3-RA)': [len(agreed_3[f]) if f in agreed_3 else 0
        for f in full_features_list],
    '4-RA': [len(agreed_4[f]) if f in agreed_4 else 0
        for f in full_features_list],
    '5-RA (FA)': [len(agreed_5[f]) if f in agreed_5 else 0
        for f in full_features_list],
})
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(S2_dermoscopic_features)

## Table of discordant diagnoses with feature overlap > DICE=0.5

In [ ]:
S3_disdiag_sp_overlap_df = pd.DataFrame(disdiag_sp_overlap)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(S3_disdiag_sp_overlap_df)

## Table of features marked by each reader (+ #SPs)

In [ ]:
S4_features_by_image_and_user = pd.DataFrame.from_dict({
    'ISIC name': image_names,
    'ISIC imageId': [img['_id'] for img in study.images],
    'diagnosis': [study.meta_data['diagnosis'][img['name']] for img in study.images],
    'ICX6 features': user_feature_stats[0],
    '6YB2 features': user_feature_stats[1],
    'PKTZ features': user_feature_stats[2],
    'Y63L features': user_feature_stats[3],
    'WTPZ features': user_feature_stats[4],
})
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(S4_features_by_image_and_user)